In [1]:
import pandas as pd 
import numpy as np
import re

In [2]:
data=pd.read_csv('insuranceFraud.csv')

In [3]:
data=data.replace('?',np.nan)

In [4]:
data.shape[1]//3

13

In [5]:
data.dropna(thresh=data.shape[1]//3, inplace=True)

In [6]:
data

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,10/17/2014,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,6/27/2006,IN,250/500,2000,1197.22,5000000,468176,...,0,NaN,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,9/6/2000,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,5/25/1990,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,6/6/2014,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,941851,7/16/1991,OH,500/1000,1000,1310.80,0,431289,...,1,NaN,87200,17440,8720,61040,Honda,Accord,2006,N
996,285,41,186934,1/5/2014,IL,100/300,1000,1436.79,0,608177,...,3,NaN,108480,18080,18080,72320,Volkswagen,Passat,2015,N
997,130,34,918516,2/17/2003,OH,250/500,500,1383.49,3000000,442797,...,3,YES,67500,7500,7500,52500,Suburu,Impreza,1996,N
998,458,62,533940,11/18/2011,IL,500/1000,2000,1356.92,5000000,441714,...,1,YES,46980,5220,5220,36540,Audi,A5,1998,N


In [7]:
data['active_days'] = abs(pd.to_datetime(data['policy_bind_date'], format='%m/%d/%Y')-pd.to_datetime(data['incident_date'], format='%m/%d/%Y')).dt.days


In [9]:
abs(pd.to_datetime(data['policy_bind_date'], format='%m/%d/%Y')-pd.to_datetime(data['incident_date'], format='%m/%d/%Y')).dt.days

0       100
1      3130
2      5282
3      8996
4       256
       ... 
995    8622
996     384
997    4358
998    1196
999    6681
Length: 1000, dtype: int64

In [10]:
col_to_delete = ['policy_number','incident_date','policy_bind_date', 'insured_zip', 'insured_hobbies','policy_state', 
                 'incident_location', 'incident_state', 'incident_city', 'auto_make' , 'auto_model' ,'insured_education_level','insured_occupation'  ]

In [11]:
data.drop(columns=col_to_delete,inplace=True)

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import KNNImputer

In [13]:
num_df = data.select_dtypes(exclude=['object']).copy()
cat_df = data.select_dtypes(include=['object']).copy()

In [23]:
cat_df.columns

Index(['policy_csl', 'insured_sex', 'insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'property_damage', 'police_report_available', 'fraud_reported'],
      dtype='object')

In [14]:
num_df

,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,active_days
0,328,48,1000,1406.91,0,53300,0,5,1,1,2,71610,6510,13020,52080,2004,100
1,228,42,2000,1197.22,5000000,0,0,8,1,0,0,5070,780,780,3510,2007,3130
2,134,29,2000,1413.14,5000000,35100,0,7,3,2,3,34650,7700,3850,23100,2007,5282
3,256,41,2000,1415.74,6000000,48900,-62400,5,1,1,2,63400,6340,6340,50720,2014,8996
4,228,44,1000,1583.91,6000000,66000,-46000,20,1,0,1,6500,1300,650,4550,2009,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,1000,1310.80,0,0,0,20,1,0,1,87200,17440,8720,61040,2006,8622
996,285,41,1000,1436.79,0,70900,0,23,1,2,3,108480,18080,18080,72320,2015,384
997,130,34,500,1383.49,3000000,35100,0,4,3,2,3,67500,7500,7500,52500,1996,4358
998,458,62,2000,1356.92,5000000,0,0,2,1,0,1,46980,5220,5220,36540,1998,1196


In [15]:

ord_categorical_cols =['insured_sex','fraud_reported', 'incident_severity' ]
ord_cat_cols_pipe = Pipeline([("impute", SimpleImputer(missing_values=np.nan, fill_value='Other', strategy='constant')),
                               ("ordenc", OrdinalEncoder())])


ohe_categorical_cols=['insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'property_damage', 'police_report_available']
ohe_cat_cols_pipe = Pipeline([("impute", SimpleImputer(missing_values=np.nan, fill_value='Other', strategy='constant')),
                               ("ohenc", OneHotEncoder() )])

from sklearn.base import BaseEstimator, TransformerMixin
class CSLSum(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # Nothing to do here
 
    def transform(self, X):
         pattern = re.compile(r'^\d+/\d+$')
         X['policy_csl']= X['policy_csl'].apply(lambda x: int(x.split('/')[0]) + int(x.split('/')[1]) if pattern.match(str(x)) else 0)
         X[X['policy_csl']==0]=X['policy_csl'].mean() 
         return X
    def get_feature_names_out(self, input_features=None):
        return ['policy_csl']

csl_pipe = Pipeline([ ("cslpipe",CSLSum()), ("stdScaler", StandardScaler() )])

class ConvertDaysInt(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # Nothing to do here
 
    def transform(self, X):
        X['active_days'] = X['active_days'].dt.days
        return X
    def get_feature_names_out(self, input_features=None):
        return ['active_days']

num_cols = list(num_df.columns)
num_cols_pipe = Pipeline([("knnimpute",  KNNImputer(n_neighbors=5)),
                               ("stdScaler", StandardScaler() )])

In [16]:
data[num_cols].dtypes

months_as_customer               int64
age                              int64
policy_deductable                int64
policy_annual_premium          float64
umbrella_limit                   int64
capital-gains                    int64
capital-loss                     int64
incident_hour_of_the_day         int64
number_of_vehicles_involved      int64
bodily_injuries                  int64
witnesses                        int64
total_claim_amount               int64
injury_claim                     int64
property_claim                   int64
vehicle_claim                    int64
auto_year                        int64
active_days                      int64
dtype: object

In [17]:
preprocessing = ColumnTransformer([
    ("ord_cat_cols_pipe", ord_cat_cols_pipe, ord_categorical_cols),
    ("ohe_cat_cols_pipe", ohe_cat_cols_pipe, ohe_categorical_cols),
    ("csl_sum", csl_pipe, ['policy_csl']), 
    ("num_pipe", num_cols_pipe, num_cols)
    ], remainder='passthrough', verbose_feature_names_out=True)

In [18]:
preprocessing2 = ColumnTransformer([
    ("ord_cat_cols_pipe", ord_cat_cols_pipe, ord_categorical_cols),
    ("ohe_cat_cols_pipe", ohe_cat_cols_pipe, ohe_categorical_cols),
    ("date_format", ConvertDaysInt(), ['active_days']),  # Apply ConvertDaysInt to 'active_days' column
    ("csl_sum", csl_pipe, ['policy_csl']),  # Apply CSLSum to 'policy_csl' column
    #("num_pipe",num_cols_pipe,num_cols)
    
], remainder='passthrough', verbose_feature_names_out=True)

In [20]:
temp = pd.DataFrame(preprocessing.fit_transform(data), columns=preprocessing.get_feature_names_out())

In [21]:
temp

,ord_cat_cols_pipe__insured_sex,ord_cat_cols_pipe__fraud_reported,ord_cat_cols_pipe__incident_severity,ohe_cat_cols_pipe__insured_relationship_husband,ohe_cat_cols_pipe__insured_relationship_not-in-family,ohe_cat_cols_pipe__insured_relationship_other-relative,ohe_cat_cols_pipe__insured_relationship_own-child,ohe_cat_cols_pipe__insured_relationship_unmarried,ohe_cat_cols_pipe__insured_relationship_wife,ohe_cat_cols_pipe__incident_type_Multi-vehicle Collision,...,num_pipe__incident_hour_of_the_day,num_pipe__number_of_vehicles_involved,num_pipe__bodily_injuries,num_pipe__witnesses,num_pipe__total_claim_amount,num_pipe__injury_claim,num_pipe__property_claim,num_pipe__vehicle_claim,num_pipe__auto_year,num_pipe__active_days
0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.956261,-0.823865,0.009759,0.461838,0.714257,-0.189283,1.165505,0.749653,-0.183440,-1.727803
1,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.524475,-0.823865,-1.210174,-1.338700,-1.807312,-1.363822,-1.372696,-1.823346,0.315491,-0.599318
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,-0.668404,1.140056,1.229693,1.362107,-0.686362,0.054644,-0.736072,-0.785565,0.315491,0.202166
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.956261,-0.823865,0.009759,0.461838,0.403135,-0.224130,-0.219722,0.677607,1.479664,1.585398
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.202666,-0.823865,-1.210174,-0.438431,-1.753121,-1.257232,-1.399654,-1.768252,0.648112,-1.669703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.202666,-0.823865,-1.210174,-0.438431,1.305049,2.051155,0.273817,1.224309,0.149180,1.446107
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.634452,-0.823865,1.229693,1.362107,2.111466,2.182342,2.214794,1.821868,1.645974,-1.622031
997,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,-1.100189,1.140056,1.229693,1.362107,0.558507,0.013648,0.020826,0.771902,-1.513924,-0.141966
998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.388046,-0.823865,-1.210174,-0.438431,-0.219110,-0.453708,-0.451976,-0.073580,-1.181303,-1.319612


In [24]:
temp['ord_cat_cols_pipe__fraud_reported']

0      1.0
1      1.0
2      0.0
3      1.0
4      0.0
      ... 
995    0.0
996    0.0
997    0.0
998    0.0
999    0.0
Name: ord_cat_cols_pipe__fraud_reported, Length: 1000, dtype: float64

In [ ]:
preprocessing.get_feature_names_out()

In [ ]:
temp.columns

In [ ]:
import seaborn as sns 

In [ ]:
sns.heatmap(temp.corr())

In [ ]:
temp['fraud_reported']

In [30]:
temp

,ord_cat_cols_pipe__insured_sex,ord_cat_cols_pipe__fraud_reported,ord_cat_cols_pipe__incident_severity,ohe_cat_cols_pipe__insured_relationship_husband,ohe_cat_cols_pipe__insured_relationship_not-in-family,ohe_cat_cols_pipe__insured_relationship_other-relative,ohe_cat_cols_pipe__insured_relationship_own-child,ohe_cat_cols_pipe__insured_relationship_unmarried,ohe_cat_cols_pipe__insured_relationship_wife,ohe_cat_cols_pipe__incident_type_Multi-vehicle Collision,...,num_pipe__incident_hour_of_the_day,num_pipe__number_of_vehicles_involved,num_pipe__bodily_injuries,num_pipe__witnesses,num_pipe__total_claim_amount,num_pipe__injury_claim,num_pipe__property_claim,num_pipe__vehicle_claim,num_pipe__auto_year,num_pipe__active_days
0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.956261,-0.823865,0.009759,0.461838,0.714257,-0.189283,1.165505,0.749653,-0.183440,-1.727803
1,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.524475,-0.823865,-1.210174,-1.338700,-1.807312,-1.363822,-1.372696,-1.823346,0.315491,-0.599318
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,-0.668404,1.140056,1.229693,1.362107,-0.686362,0.054644,-0.736072,-0.785565,0.315491,0.202166
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.956261,-0.823865,0.009759,0.461838,0.403135,-0.224130,-0.219722,0.677607,1.479664,1.585398
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.202666,-0.823865,-1.210174,-0.438431,-1.753121,-1.257232,-1.399654,-1.768252,0.648112,-1.669703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.202666,-0.823865,-1.210174,-0.438431,1.305049,2.051155,0.273817,1.224309,0.149180,1.446107
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.634452,-0.823865,1.229693,1.362107,2.111466,2.182342,2.214794,1.821868,1.645974,-1.622031
997,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,-1.100189,1.140056,1.229693,1.362107,0.558507,0.013648,0.020826,0.771902,-1.513924,-0.141966
998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.388046,-0.823865,-1.210174,-0.438431,-0.219110,-0.453708,-0.451976,-0.073580,-1.181303,-1.319612


In [31]:
X_train = temp.drop(columns=['ord_cat_cols_pipe__fraud_reported'])
y_train = temp['ord_cat_cols_pipe__fraud_reported']

In [33]:
from sklearn.model_selection import GridSearchCV

In [34]:
from xgboost import XGBClassifier

In [35]:
xgb=XGBClassifier()
y_pred = xgb.fit(X_train, y_train).predict(X_train)

In [36]:
ac2=accuracy_score(y_train,y_pred)
ac2

NameError: name 'accuracy_score' is not defined

In [37]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [38]:
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Precision: 1.00
Recall: 1.00
F1 Score: 1.00


In [39]:
param_grid = {"n_estimators": [10, 50, 100, 130], "criterion": ['gini', 'entropy'],
                               "max_depth": range(2, 10, 1)}

            #Creating an object of the Grid Search class
grid = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5,  verbose=3,n_jobs=-1)

In [40]:
#finding the best parameters
grid.fit(X_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


C:\Users\mural\anaconda3\envs\tfwn\lib\site-packages\xgboost\core.py:158: UserWarning: [12:22:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "criterion" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'n_estimators': [10, 50, 100, 130]},
             verbose=3)

In [41]:
grid.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, criterion='gini', device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=10, n_jobs=None,
              num_parallel_tree=None, ...)

In [43]:
best_model = grid.best_estimator_

In [44]:
best_model.fit(X_train, y_train)

C:\Users\mural\anaconda3\envs\tfwn\lib\site-packages\xgboost\core.py:158: UserWarning: [12:26:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "criterion" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, criterion='gini', device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=10, n_jobs=None,
              num_parallel_tree=None, ...)

In [45]:
import joblib
joblib_file = "best_model.pkl"
joblib.dump(best_model, joblib_file)

['best_model.pkl']